In [1]:
from select import select
from odps import ODPS
from odps import options
from odps.df import DataFrame
import pandas as pd
import numpy as np
import re
import math
from collections import Counter
from collections import defaultdict

ACCESS_ID = 'LTAI5t9uwJrh5eJ7Q5E37D1s'
SECRET_ACCESS_KEY = 'NCFHOAnvqfnTrpypgR4b3cNawP8fnB'
ODPS_PROJECT = 'OpenDigger_prod_dev'
ODPS_ENDPOINT = 'http://service.cn-shanghai.maxcompute.aliyun.com/api'
# ACCESS_ID = 'LTAI5tSjrYp6JDW2bbiTdegU'
# SECRET_ACCESS_KEY = 'OE6JbSqOZUey5fzr9Wg6fuYYKvslZx'
# 创建odps实例
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY,
         project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False
# options.read_timeout = 3600000
users = DataFrame(o.get_table('ods_github_users'))
print(users.dtypes)


odps.Schema {
  login                     string        
  created_at                date          
  database_id               int64         
  location                  string        
  company                   string        
  bio                       string        
  is_employee               boolean       
  email                     string        
  infoname                  string        
  followers                 string        
  following                 string        
  time                      date          
  name                      string        
  lastupdatedat             date          
  nextupdateat              date          
}



In [ ]:
def exe_sql(sql):
    data = []
    with o.execute_sql(sql).open_reader() as reader:
        d = defaultdict(list)  
        for record in reader:
            for res in record:
                d[res[0]].append(res[1])  
        data = pd.DataFrame.from_dict(d,orient='index').T 
    return data

rdata =  exe_sql("SELECT * FROM ods_github_users;")

In [ ]:
data = rdata.dropna(axis='index', how='all', subset=['followers'])
data = data.drop_duplicates(['database_id'])  
data

In [1]:
import community

import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

In [5]:
cnt = 0
for index, row in data.iterrows():
    follower_list = row['followers'].split(',')
    for follower in follower_list:
         G.add_edge(follower, row['database_id'])
    cnt = cnt + 1
    if cnt == 1000:
        break

In [ ]:
nx.write_edgelist(G,"community.txt",data = False)

In [2]:
G = nx.read_edgelist('community.txt')

In [3]:
partition = community.best_partition(G)

In [8]:
# communities = []
# for p in partition:
#         communities = []
#         for vertices in self._cid_vertices.values():
#             if len(vertices) != 0:
#                 c = set()
#                 for vid in vertices:
#                     c.update(self._vid_vertex[vid]._nodes)
#                 communities.append(c)


In [11]:
partition = sorted(partition.items(), key=lambda item:item[1])

In [21]:
import pandas as pd
data = pd.DataFrame(partition)

In [27]:

data = data.rename(columns = {0:'user_id', 1:'community'})

In [30]:
data

,user_id,community
0,86969786,0
1,60898417,0
2,191335,1
3,57493545,1
4,31819732,1
...,...,...
192974,30351955,283
192975,964724,284
192976,24510492,284
192977,26847367,285


In [31]:
df = df.rename(columns = {'user_id':'count'})

In [32]:
df

,count
community,
0,2
1,2417
2,2152
3,5105
4,2
...,...
281,5
282,2
283,2


In [ ]:
size = float(len(set(partition.values())))
pos = nx.spring_layout(G)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(G, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(G,pos, alpha=0.5)